# Kaggle Learn-and-compete `Poker Rule Induction`

## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from functools import reduce
from itertools import permutations

#### Read data

In [2]:
test_dataset = pd.read_csv('test.csv', delimiter=',')

In [3]:
test_dataset

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
0,1,1,10,2,2,3,3,3,8,1,1
1,2,2,13,3,5,3,7,4,6,1,4
2,3,1,3,1,11,2,8,2,1,2,4
3,4,1,6,3,3,4,7,1,8,3,11
4,5,2,10,3,4,1,6,2,12,2,6
5,6,1,4,3,10,2,11,2,6,1,7
6,7,1,10,3,8,1,4,3,11,3,9
7,8,2,11,3,8,1,1,1,11,2,3
8,9,3,4,1,1,1,3,3,5,3,6
9,10,3,12,2,1,1,3,1,2,3,10


In [4]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [5]:
train_dataset['hand'].value_counts()

0    12493
1    10599
2     1206
3      513
4       93
5       54
6       36
7        6
9        5
8        5
Name: hand, dtype: int64

##### Data augmentation

In [6]:
cards = ['C1','C2','C3','C4','C5']
suits = ['S1','S2','S3','S4','S5']

def shuffle_columns(data, columns):
    shuffled = data[columns].reindex(np.random.permutation(data[columns].columns), axis=1)
    return pd.concat([shuffled, data.drop(columns, axis=1)], axis=1)

def repeat(data, times):
    repeated = data.copy()
    for i in range(times - 1):
        repeated = pd.concat([shuffle_columns(repeated, cards), data], axis=0)
    return repeated

def data_augmentation(data):
    data_aug = pd.concat([data[cards], data[suits], data['hand']], axis=1)
    counts = data['hand'].value_counts()
    for value, count in enumerate(counts):
        if value == 0:
            continue
        else:
            times = counts[0]//count
            
        if (times == 1):
            continue
            
        repeated = repeat(data[data['hand'] == value], times)
        data_aug = pd.concat([data_aug, repeated], axis=0)
    data_aug.index = list(range(data_aug.shape[0]))
    return data_aug

In [7]:
train_dataset_aug = data_augmentation(train_dataset)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [8]:
train_dataset_aug['hand'].value_counts()

2    13266
3    12825
4    12555
6    12528
5    12528
7    12498
9    12495
8    12495
0    12493
1    10599
Name: hand, dtype: int64

In [9]:
train_dataset_aug.shape

(124282, 11)

In [10]:
data = train_dataset_aug[np.concatenate((cards, suits))]
labels = train_dataset_aug[['hand']]

In [11]:
data[0:10]

,C1,C2,C3,C4,C5,S1,S2,S3,S4,S5
0,9,1,2,7,8,4,2,2,4,2
1,4,6,12,11,7,1,3,1,3,2
2,11,1,7,11,1,1,4,3,4,2
3,9,4,6,9,9,2,2,3,1,4
4,8,4,11,2,1,1,2,2,2,2
5,5,5,13,3,13,2,1,2,2,3
6,10,6,4,13,5,3,4,1,2,4
7,10,1,13,2,7,4,3,2,4,4
8,2,10,3,4,9,3,4,3,4,1
9,7,8,8,13,12,2,3,4,2,2


In [12]:
labels_vect = tf.keras.utils.to_categorical(labels)

#### One hot encoding

In [13]:
def data_hotencode(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

def hand_hotencode(hand, columns, encoder):
    df = pd.DataFrame(data=[hand], columns=['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5'])
    return data_hotencode(df, columns, encoder, 'transform')

In [14]:
encoder = OneHotEncoder()
columns_to_hotencode = ['S1', 'S2', 'S3', 'S4', 'S5']
data_encoded = data_hotencode(data, columns_to_hotencode, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(124282, 25)


,C1,C2,C3,C4,C5,0,1,2,3,4,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [15]:
def normalize(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data.drop(columns, axis=1), data_normalized], axis=1)
    to_return.columns = list(range(to_return.shape[1]))
    return to_return

In [16]:
columns_to_normalize = ['C1', 'C2', 'C3', 'C4', 'C5']
mean = data_encoded[columns_to_normalize].mean().mean()
std = pd.Series(data_encoded[columns_to_normalize].values.flatten()).std()
data_normalized = normalize(data_encoded, columns_to_normalize, mean, std)
data_normalized.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.489357,-1.615549,-1.352436,-0.036870,0.226243
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,-0.826210,-0.299983,1.278696,1.015583,-0.036870
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.015583,-1.615549,-0.036870,1.015583,-1.615549
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.489357,-0.826210,-0.299983,0.489357,0.489357
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.226243,-0.826210,1.015583,-1.352436,-1.615549


In [17]:
data_normalized.shape

(124282, 25)

#### Test function

In [18]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [19]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


### Solutions

1. Neural network
2. Random forest

Ousiders:
 - Genetic algorithm
 - Decision tree

### Neural Network

In [20]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(200, activation='relu', input_shape=(25,)))
#model.add(keras.layers.Dropout(0.1))
#hidden
model.add(keras.layers.Dense(400, activation='relu'))
#model.add(keras.layers.Dropout(0.05))
model.add(keras.layers.Dense(200, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=500, batch_size=512)

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 30us/step - loss: 0.9602 - acc: 0.6440

Epoch 2/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 28us/step - loss: 0.5164 - acc: 0.7785

Epoch 3/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 28us/step - loss: 0.4415 - acc: 0.8125

Epoch 4/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 29us/step - loss: 0.3798 - acc: 0.8401

Epoch 5/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 29us/step - loss: 0.3294 - acc: 0.8595

Epoch 6/500
124282/124282 [==============================]124282/124282 [==============================]

124282/124282 [==============================]124282/124282 [==============================] - 4s 31us/step - loss: 0.0992 - acc: 0.9506

Epoch 52/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 31us/step - loss: 0.0956 - acc: 0.9536

Epoch 53/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 29us/step - loss: 0.0948 - acc: 0.9539

Epoch 54/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 33us/step - loss: 0.1141 - acc: 0.9466

Epoch 55/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 30us/step - loss: 0.0951 - acc: 0.9541

Epoch 56/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 27us/step - loss: 0.0937 - acc: 0.9552

Epoch 57/500
124282/124282 [==============================]124282/124282 [==============================] -

124282/124282 [==============================]124282/124282 [==============================] - 3s 25us/step - loss: 0.0253 - acc: 0.9914

Epoch 106/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 25us/step - loss: 0.0226 - acc: 0.9926

Epoch 107/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 25us/step - loss: 0.0245 - acc: 0.9915

Epoch 108/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 25us/step - loss: 0.0366 - acc: 0.9860

Epoch 109/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 25us/step - loss: 0.0385 - acc: 0.9855

Epoch 110/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 25us/step - loss: 0.0314 - acc: 0.9883

Epoch 111/500
124282/124282 [==============================]124282/124282 [===========================


Epoch 159/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 27us/step - loss: 0.0512 - acc: 0.9836

Epoch 160/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 27us/step - loss: 0.0338 - acc: 0.9881

Epoch 161/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 27us/step - loss: 0.0096 - acc: 0.9970

Epoch 162/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 31us/step - loss: 0.0041 - acc: 0.9994

Epoch 163/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 33us/step - loss: 0.0022 - acc: 0.9999

Epoch 164/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 30us/step - loss: 0.0016 - acc: 1.0000

Epoch 165/500
124282/124282 [==============================]124282/124282 [============

...

124282/124282 [==============================]124282/124282 [==============================] - 3s 27us/step - loss: 0.0095 - acc: 0.9974

Epoch 266/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 27us/step - loss: 0.0789 - acc: 0.9755

Epoch 267/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 27us/step - loss: 0.0106 - acc: 0.9967

Epoch 268/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 27us/step - loss: 0.0034 - acc: 0.9995

Epoch 269/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 27us/step - loss: 0.0014 - acc: 0.9999

Epoch 270/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 27us/step - loss: 7.3736e-04 - acc: 1.0000

Epoch 271/500
124282/124282 [==============================]124282/124282 [=======================

124282/124282 [==============================]124282/124282 [==============================] - 4s 31us/step - loss: 0.0562 - acc: 0.9823

Epoch 319/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 35us/step - loss: 0.0142 - acc: 0.9951

Epoch 320/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 29us/step - loss: 0.0040 - acc: 0.9990

Epoch 321/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 30us/step - loss: 0.0011 - acc: 0.9999

Epoch 322/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 32us/step - loss: 5.0911e-04 - acc: 1.0000

Epoch 323/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 32us/step - loss: 4.0151e-04 - acc: 1.0000

Epoch 324/500
124282/124282 [==============================]124282/124282 [===================

124282/124282 [==============================]124282/124282 [==============================] - 5s 39us/step - loss: 0.0072 - acc: 0.9977

Epoch 372/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 29us/step - loss: 0.0022 - acc: 0.9996

Epoch 373/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 33us/step - loss: 0.0012 - acc: 0.9998

Epoch 374/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 29us/step - loss: 6.0756e-04 - acc: 1.0000

Epoch 375/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 28us/step - loss: 3.8852e-04 - acc: 1.0000

Epoch 376/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 26us/step - loss: 0.0028 - acc: 0.9992

Epoch 377/500
124282/124282 [==============================]124282/124282 [===================

124282/124282 [==============================]124282/124282 [==============================] - 4s 29us/step - loss: 1.5165e-04 - acc: 1.0000

Epoch 425/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 28us/step - loss: 1.3697e-04 - acc: 1.0000

Epoch 426/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 29us/step - loss: 1.2378e-04 - acc: 1.0000

Epoch 427/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 28us/step - loss: 1.1227e-04 - acc: 1.0000

Epoch 428/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 28us/step - loss: 1.0185e-04 - acc: 1.0000

Epoch 429/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 28us/step - loss: 9.2215e-05 - acc: 1.0000

Epoch 430/500
124282/124282 [==============================]124282/124282 [===

124282/124282 [==============================]124282/124282 [==============================] - 3s 28us/step - loss: 8.2089e-04 - acc: 1.0000

Epoch 478/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 28us/step - loss: 5.7016e-04 - acc: 1.0000

Epoch 479/500
124282/124282 [==============================]124282/124282 [==============================] - 3s 28us/step - loss: 5.0159e-04 - acc: 1.0000

Epoch 480/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 29us/step - loss: 5.7674e-04 - acc: 0.9999

Epoch 481/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 28us/step - loss: 4.1107e-04 - acc: 1.0000

Epoch 482/500
124282/124282 [==============================]124282/124282 [==============================] - 4s 28us/step - loss: 2.5669e-04 - acc: 1.0000

Epoch 483/500
124282/124282 [==============================]124282/124282 [===

In [21]:
y_test = test_dataset.drop(['id'], axis=1).apply(calc_hand_label, axis=1)

In [39]:
model.save('./model-sort-166')

### Predictions

In [22]:
test_dataset_noid = test_dataset.drop(['id'], axis=1)

In [23]:
test_sample = pd.concat([test_dataset_noid[cards], test_dataset_noid[suits]], axis=1)
test_sample.head()

,C1,C2,C3,C4,C5,S1,S2,S3,S4,S5
0,10,2,3,8,1,1,2,3,3,1
1,13,5,7,6,4,2,3,3,4,1
2,3,11,8,1,4,1,1,2,2,2
3,6,3,7,8,11,1,3,4,1,3
4,10,4,6,12,6,2,3,1,2,2


In [24]:
test_data_sample_encoded = normalize(
    data_hotencode(test_sample, columns_to_hotencode, encoder),
    columns_to_normalize,
    mean,
    std
)

In [1]:
model1 = keras.models.load_model('./model-sort-166')
model1.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

NameError: name 'keras' is not defined

In [25]:
test_data_sample_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.752470,-1.352436,-1.089323,0.226243,-1.615549
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.541809,-0.563096,-0.036870,-0.299983,-0.826210
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,-1.089323,1.015583,0.226243,-1.615549,-0.826210
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,-0.299983,-1.089323,-0.036870,0.226243,1.015583
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.752470,-0.826210,-0.299983,1.278696,-0.299983


In [26]:
test_data_sample_predicted_10 = model.predict(np.array(test_data_sample_encoded))
test_data_sample_predicted = np.array(list(map(np.argmax, test_data_sample_predicted_10)))

In [27]:
#test_data_sample_predicted = np.maximum(test_data_sample_predicted_1, test_data_sample_predicted_2)
#test_data_sample_predicted = test_data_sample_predicted_2

test_data_sample_labels = y_test
df = pd.DataFrame({
    'predicted': test_data_sample_predicted, 
    'real': test_data_sample_labels
})
df = pd.concat([test_sample, df], axis=1)
wrong_answers = df[df['predicted'] != df['real']]
print(wrong_answers.shape)
wrong_answers

(488797, 12)


,C1,C2,C3,C4,C5,S1,S2,S3,S4,S5,predicted,real
1,13,5,7,6,4,2,3,3,4,1,1,0
3,6,3,7,8,11,1,3,4,1,3,1,0
4,10,4,6,12,6,2,3,1,2,2,0,1
6,10,8,4,11,9,1,3,1,3,3,1,0
8,4,1,3,5,6,3,1,1,3,3,1,0
9,12,1,3,2,10,3,2,1,1,3,1,0
10,7,1,8,10,11,1,3,4,4,3,1,0
13,7,2,3,4,1,4,4,3,1,1,1,0
14,8,10,11,5,12,4,3,4,3,1,1,0
17,7,6,4,12,6,3,1,1,3,3,0,1


In [73]:
df[df['real'] == 1].shape

(422498, 12)

In [68]:
wrong_answers['real'].value_counts()

1    214857
0    203935
2     44101
3     19954
4      3809
6      1414
5       733
7       230
8        12
9         2
Name: real, dtype: int64

In [48]:
wrong_answers.drop(['predicted', 'real'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

34        K♣, Q♣, 10♦, A♦, J♥
665        4♣, 5♣, 3♣, A♣, 2♣
3902       4♠, 3♦, 2♣, A♣, 5♥
9503       4♥, A♠, 2♠, 5♣, 3♣
12027     K♦, J♦, 10♦, Q♥, A♥
14357      5♣, A♠, 3♠, 4♠, 2♣
14657      A♠, 5♥, 4♣, 3♣, 2♥
15059     K♦, Q♥, A♥, 10♥, J♠
15877     10♦, Q♥, K♥, A♥, J♠
20151      4♣, A♣, 3♣, 5♦, 2♦
36911     K♦, J♣, A♣, 10♦, Q♥
42427      3♦, A♣, 2♣, 5♥, 4♣
48362     Q♦, J♦, A♠, 10♥, K♦
50832     Q♦, 10♥, A♦, K♥, J♠
51428      A♦, 4♣, 5♠, 3♥, 2♥
57633     J♠, Q♦, K♠, 10♠, A♠
65236     10♥, Q♣, A♦, K♦, J♠
66296     K♠, Q♣, J♥, 10♦, A♥
69065     Q♣, J♥, A♦, 10♦, K♦
70575     9♣, 8♥, K♦, J♠, 10♠
72500      3♠, 5♠, 2♣, A♣, 4♥
78123     10♦, A♥, J♥, K♣, Q♥
85179     Q♦, K♣, A♣, J♦, 10♥
88839     10♣, J♥, Q♦, A♥, K♥
91839      5♣, A♣, 4♠, 3♣, 2♦
95994     10♣, K♥, Q♥, J♠, A♦
96353     J♣, 9♥, 10♦, K♦, 8♠
97192      5♠, 3♣, 4♠, 2♦, A♣
100764     3♣, 5♣, 2♣, 4♥, A♣
103163    K♥, A♣, Q♦, J♣, 10♥
                 ...         
879365    K♣, A♥, 10♥, Q♦, J♦
890194    9♣, J♥, Q♣, 10♣, A♥
893648    